In [1]:
import os
import time
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
# Wait for process_data to finish

while 'experiment_data__20_steps__60_days.hkl' not in os.listdir('data/processed_data'):
    time.sleep(600)

In [3]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [4]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [5]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    action_testing_input = data['action_testing_input']
    recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    action_model = Dropout(rate=0.5)(action_input_layer)
    action_model_hook = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    action_model = Dropout(rate=0.5)(action_model_hook)
    
    action_completion_output_layer = Dense(units=1, activation='sigmoid', name='action_completion')(action_model)
    action_problems_output_layer = Dense(units=1, activation='linear', name='action_problems')(action_model)
    
    action_model = Model(action_input_layer, [action_completion_output_layer, action_problems_output_layer])
    action_model.compile(optimizer='adam', loss={'action_completion': 'binary_crossentropy', 'action_problems': 'mse'})
    
    
    # Recurrent Model
    recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    recurrent_model = Dropout(rate=0.5)(recurrent_model)
    recurrent_model_hook = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    recurrent_model = Dropout(rate=0.5)(recurrent_model_hook)
    
    recurrent_completion_output_layer = Dense(units=1, activation='sigmoid', name='recurrent_completion')(recurrent_model)
    recurrent_problems_output_layer = Dense(units=1, activation='linear', name='recurrent_problems')(recurrent_model)
    
    recurrent_model = Model(recurrent_input_layer, [recurrent_completion_output_layer, recurrent_problems_output_layer])
    recurrent_model.compile(optimizer='adam', loss={'recurrent_completion': 'binary_crossentropy', 'recurrent_problems': 'mse'})
    
    
    # Prior Model
    prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    prior_model = Dropout(rate=0.5)(prior_input_layer)
    prior_model_hook = Dense(units=64, activation='sigmoid')(prior_model)
    prior_model = Dropout(rate=0.5)(prior_model_hook)
    
    prior_completion_output_layer = Dense(units=1, activation='sigmoid', name='prior_completion')(prior_model)
    prior_problems_output_layer = Dense(units=1, activation='linear', name='prior_problems')(prior_model)
    
    prior_model = Model(prior_input_layer, [prior_completion_output_layer, prior_problems_output_layer])
    prior_model.compile(optimizer='adam', loss={'prior_completion': 'binary_crossentropy', 'prior_problems': 'mse'})
    
    
    # Combined Model
    combined_model = Concatenate()([action_model_hook, recurrent_model_hook, prior_model_hook])
    combined_model = Dropout(rate=0.5)(combined_model)
    combined_completion_output_layer = Dense(units=1, activation='sigmoid', name='combined_completion')(combined_model)
    combined_problems_output_layer = Dense(units=1, activation='linear', name='combined_problems')(combined_model)
    
    combined_model = Model([action_input_layer, recurrent_input_layer, prior_input_layer], [combined_completion_output_layer, combined_problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'combined_completion': 'binary_crossentropy', 'combined_problems': 'mse'})
    
    
    # Train Models
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    
    
    # Train Partial Models
    weights = {'action_completion': np.ones_like(completion_training_target) * 16, 'action_problems': completion_training_target}
    action_model.fit(x={'action': action_training_input},
                     y={'action_completion': completion_training_target, 'action_problems': problems_training_target},
                     epochs=1000,
                     validation_split=0.25,
                     callbacks=es,
                     sample_weight=weights,
                     verbose=1)
    
    completion_testing_output, problems_testing_output = action_model.predict({'action': action_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          np.array(['action'] * testing_target_sequence.size), 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'model', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    
    
    weights = {'recurrent_completion': np.ones_like(completion_training_target) * 16, 'recurrent_problems': completion_training_target}
    recurrent_model.fit(x={'recurrent': recurrent_training_input},
                       y={'recurrent_completion': completion_training_target, 'recurrent_problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    completion_testing_output, problems_testing_output = recurrent_model.predict({'recurrent': recurrent_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          np.array(['assignment'] * testing_target_sequence.size), 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'model', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    
    
    weights = {'prior_completion': np.ones_like(completion_training_target) * 16, 'prior_problems': completion_training_target}
    prior_model.fit(x={'prior': prior_training_input},
                       y={'prior_completion': completion_training_target, 'prior_problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    completion_testing_output, problems_testing_output = prior_model.predict({'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          np.array(['student'] * testing_target_sequence.size), 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'model', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    
    weights = {'combined_completion': np.ones_like(completion_training_target) * 16, 'combined_problems': completion_training_target}
    combined_model.fit(x={'action': action_training_input, 'recurrent': recurrent_training_input, 'prior': prior_training_input},
                       y={'combined_completion': completion_training_target, 'combined_problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'action': action_testing_input, 
                                                                                 'recurrent': recurrent_testing_input, 
                                                                                 'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          np.array(['combined'] * testing_target_sequence.size), 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'model', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)

pd.concat(results).to_csv('cross_validation_results.csv', index=False)

Epoch 1/1000
2862/2862 [==============================] - 157s 54ms/step - loss: 25.4099 - action_completion_loss: 9.1352 - action_problems_loss: 16.2747 - val_loss: 15.1004 - val_action_completion_loss: 9.8268 - val_action_problems_loss: 5.2737
Epoch 2/1000
2862/2862 [==============================] - 152s 53ms/step - loss: 16.6198 - action_completion_loss: 8.8854 - action_problems_loss: 7.7343 - val_loss: 15.0232 - val_action_completion_loss: 9.8172 - val_action_problems_loss: 5.2060
Epoch 3/1000
2862/2862 [==============================] - 149s 52ms/step - loss: 16.2921 - action_completion_loss: 8.8219 - action_problems_loss: 7.4702 - val_loss: 14.7759 - val_action_completion_loss: 9.6253 - val_action_problems_loss: 5.1505
Epoch 4/1000
2862/2862 [==============================] - 152s 53ms/step - loss: 16.1135 - action_completion_loss: 8.7622 - action_problems_loss: 7.3514 - val_loss: 14.6873 - val_action_completion_loss: 9.5879 - val_action_problems_loss: 5.0994
Epoch 5/1000
2862/2

Epoch 11/1000
2862/2862 [==============================] - 86s 30ms/step - loss: 14.7602 - recurrent_completion_loss: 7.7378 - recurrent_problems_loss: 7.0225 - val_loss: 13.7798 - val_recurrent_completion_loss: 8.7815 - val_recurrent_problems_loss: 4.9983
Epoch 12/1000
2862/2862 [==============================] - 87s 30ms/step - loss: 14.7536 - recurrent_completion_loss: 7.7340 - recurrent_problems_loss: 7.0195 - val_loss: 13.6686 - val_recurrent_completion_loss: 8.6750 - val_recurrent_problems_loss: 4.9937
Epoch 13/1000
2862/2862 [==============================] - 87s 30ms/step - loss: 14.7370 - recurrent_completion_loss: 7.7444 - recurrent_problems_loss: 6.9926 - val_loss: 13.6491 - val_recurrent_completion_loss: 8.6665 - val_recurrent_problems_loss: 4.9826
Epoch 14/1000
2862/2862 [==============================] - 86s 30ms/step - loss: 14.7198 - recurrent_completion_loss: 7.7355 - recurrent_problems_loss: 6.9843 - val_loss: 13.7065 - val_recurrent_completion_loss: 8.7017 - val_recu

2862/2862 [==============================] - 4s 2ms/step - loss: 15.1167 - prior_completion_loss: 8.1334 - prior_problems_loss: 6.9833 - val_loss: 13.7800 - val_prior_completion_loss: 8.8419 - val_prior_problems_loss: 4.9381
Epoch 32/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.1200 - prior_completion_loss: 8.1345 - prior_problems_loss: 6.9855 - val_loss: 13.7558 - val_prior_completion_loss: 8.8074 - val_prior_problems_loss: 4.9484
Epoch 33/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.1114 - prior_completion_loss: 8.1278 - prior_problems_loss: 6.9836 - val_loss: 13.7701 - val_prior_completion_loss: 8.8308 - val_prior_problems_loss: 4.9394
Epoch 34/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.1251 - prior_completion_loss: 8.1227 - prior_problems_loss: 7.0024 - val_loss: 13.7646 - val_prior_completion_loss: 8.8240 - val_prior_problems_loss: 4.9406
Epoch 35/1000
2862/2862 [==============================] -

2862/2862 [==============================] - 157s 55ms/step - loss: 15.4699 - action_completion_loss: 8.8063 - action_problems_loss: 6.6636 - val_loss: 14.8517 - val_action_completion_loss: 9.9539 - val_action_problems_loss: 4.8979
Epoch 16/1000
2862/2862 [==============================] - 157s 55ms/step - loss: 15.4652 - action_completion_loss: 8.8013 - action_problems_loss: 6.6639 - val_loss: 14.8567 - val_action_completion_loss: 9.9599 - val_action_problems_loss: 4.8969
Epoch 17/1000
2862/2862 [==============================] - 158s 55ms/step - loss: 15.4658 - action_completion_loss: 8.8029 - action_problems_loss: 6.6629 - val_loss: 14.8256 - val_action_completion_loss: 9.9294 - val_action_problems_loss: 4.8962
Epoch 18/1000
2862/2862 [==============================] - 158s 55ms/step - loss: 15.4441 - action_completion_loss: 8.7885 - action_problems_loss: 6.6555 - val_loss: 14.8191 - val_action_completion_loss: 9.9231 - val_action_problems_loss: 4.8960
Epoch 19/1000
2862/2862 [=====

Epoch 15/1000
2862/2862 [==============================] - 82s 29ms/step - loss: 14.1262 - recurrent_completion_loss: 7.7060 - recurrent_problems_loss: 6.4202 - val_loss: 13.5619 - val_recurrent_completion_loss: 8.7566 - val_recurrent_problems_loss: 4.8053
Epoch 16/1000
2862/2862 [==============================] - 82s 29ms/step - loss: 14.1098 - recurrent_completion_loss: 7.6877 - recurrent_problems_loss: 6.4221 - val_loss: 13.5582 - val_recurrent_completion_loss: 8.7812 - val_recurrent_problems_loss: 4.7770
Epoch 17/1000
2862/2862 [==============================] - 82s 29ms/step - loss: 14.1100 - recurrent_completion_loss: 7.6782 - recurrent_problems_loss: 6.4318 - val_loss: 13.6425 - val_recurrent_completion_loss: 8.8592 - val_recurrent_problems_loss: 4.7833
Epoch 18/1000
2862/2862 [==============================] - 83s 29ms/step - loss: 14.0921 - recurrent_completion_loss: 7.6753 - recurrent_problems_loss: 6.4168 - val_loss: 13.6219 - val_recurrent_completion_loss: 8.8356 - val_recu

...

Epoch 5/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.2038 - prior_completion_loss: 8.1487 - prior_problems_loss: 7.0551 - val_loss: 13.7095 - val_prior_completion_loss: 8.8444 - val_prior_problems_loss: 4.8651
Epoch 6/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.1758 - prior_completion_loss: 8.1283 - prior_problems_loss: 7.0475 - val_loss: 13.7194 - val_prior_completion_loss: 8.8453 - val_prior_problems_loss: 4.8741
Epoch 7/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.1277 - prior_completion_loss: 8.1269 - prior_problems_loss: 7.0008 - val_loss: 13.7535 - val_prior_completion_loss: 8.8535 - val_prior_problems_loss: 4.9000
Epoch 8/1000
2862/2862 [==============================] - 4s 2ms/step - loss: 15.0812 - prior_completion_loss: 8.1233 - prior_problems_loss: 6.9579 - val_loss: 13.7320 - val_prior_completion_loss: 8.8629 - val_prior_problems_loss: 4.8691
Epoch 9/1000
2862/2862 [========================

Epoch 11/1000
2862/2862 [==============================] - 173s 61ms/step - loss: 19.7726 - combined_completion_loss: 9.3377 - combined_problems_loss: 10.4349 - val_loss: 3083.2097 - val_combined_completion_loss: 134.6946 - val_combined_problems_loss: 2948.5161
Epoch 12/1000
2862/2862 [==============================] - 173s 60ms/step - loss: 15.8237 - combined_completion_loss: 8.0290 - combined_problems_loss: 7.7947 - val_loss: 934.6948 - val_combined_completion_loss: 45.7613 - val_combined_problems_loss: 888.9335
Epoch 13/1000
2862/2862 [==============================] - 173s 60ms/step - loss: 14.7989 - combined_completion_loss: 7.6795 - combined_problems_loss: 7.1193 - val_loss: 288.1018 - val_combined_completion_loss: 50.6944 - val_combined_problems_loss: 237.4075
Epoch 14/1000
2862/2862 [==============================] - 173s 60ms/step - loss: 14.3630 - combined_completion_loss: 7.5404 - combined_problems_loss: 6.8226 - val_loss: 128.5477 - val_combined_completion_loss: 21.3756 - v

Epoch 3/1000
2862/2862 [==============================] - 87s 30ms/step - loss: 15.2079 - recurrent_completion_loss: 7.8727 - recurrent_problems_loss: 7.3352 - val_loss: 13.3905 - val_recurrent_completion_loss: 8.3716 - val_recurrent_problems_loss: 5.0188
Epoch 4/1000
2862/2862 [==============================] - 87s 30ms/step - loss: 15.1114 - recurrent_completion_loss: 7.8385 - recurrent_problems_loss: 7.2729 - val_loss: 13.3576 - val_recurrent_completion_loss: 8.3563 - val_recurrent_problems_loss: 5.0013
Epoch 5/1000
2862/2862 [==============================] - 87s 31ms/step - loss: 15.0351 - recurrent_completion_loss: 7.8215 - recurrent_problems_loss: 7.2136 - val_loss: 13.3471 - val_recurrent_completion_loss: 8.3659 - val_recurrent_problems_loss: 4.9813
Epoch 6/1000
2862/2862 [==============================] - 87s 30ms/step - loss: 14.9664 - recurrent_completion_loss: 7.8022 - recurrent_problems_loss: 7.1642 - val_loss: 13.3472 - val_recurrent_completion_loss: 8.3477 - val_recurren

Epoch 13/1000
2862/2862 [==============================] - 5s 2ms/step - loss: 15.2678 - prior_completion_loss: 8.1399 - prior_problems_loss: 7.1279 - val_loss: 13.6026 - val_prior_completion_loss: 8.6164 - val_prior_problems_loss: 4.9862
Epoch 14/1000
2862/2862 [==============================] - 5s 2ms/step - loss: 15.2387 - prior_completion_loss: 8.1286 - prior_problems_loss: 7.1102 - val_loss: 13.5818 - val_prior_completion_loss: 8.6017 - val_prior_problems_loss: 4.9801
Epoch 15/1000
2862/2862 [==============================] - 5s 2ms/step - loss: 15.2358 - prior_completion_loss: 8.1360 - prior_problems_loss: 7.0998 - val_loss: 13.5847 - val_prior_completion_loss: 8.6066 - val_prior_problems_loss: 4.9780
Epoch 16/1000
2862/2862 [==============================] - 5s 2ms/step - loss: 15.1978 - prior_completion_loss: 8.1197 - prior_problems_loss: 7.0781 - val_loss: 13.6080 - val_prior_completion_loss: 8.6319 - val_prior_problems_loss: 4.9761
Epoch 17/1000
2862/2862 [===================

Epoch 12/1000
2862/2862 [==============================] - 182s 63ms/step - loss: 14.3815 - combined_completion_loss: 7.5349 - combined_problems_loss: 6.8466 - val_loss: 13.3269 - val_combined_completion_loss: 8.3003 - val_combined_problems_loss: 5.0267
Epoch 13/1000
2862/2862 [==============================] - 182s 64ms/step - loss: 14.3658 - combined_completion_loss: 7.5130 - combined_problems_loss: 6.8528 - val_loss: 13.2921 - val_combined_completion_loss: 8.2907 - val_combined_problems_loss: 5.0015


In [6]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('cross_validation_results.csv')

In [7]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []

# Calculate the metrics for each sequence
for keys, df in results.groupby(['model', 'target_sequence']):
    
    model, sequence = keys
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([model, 
                    sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for keys, df in results.groupby(['model', 'fold']):
    
    model, fold = keys
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([model,
                    f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
for model, df in results.groupby('model'):

    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)

    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([model,
                    'all_target_data', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['model',
                                'group', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('cross_validation_metrics.csv', index=False)

In [8]:
metrics.iloc[-10:]

,model,group,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
1162,combined,fold_4,None,30524,0.777088,0.764513,0.197169,0.198017,0.161192,1.897079,0.194266,0.052319,0.052508,7.548419
1163,student,fold_0,None,30525,0.745143,0.766749,0.142473,0.142627,0.161608,1.931005,0.211434,0.035382,0.035408,7.509819
1164,student,fold_1,None,30524,0.742713,0.768084,0.142689,0.143926,0.160928,1.948399,0.209259,0.029555,0.029662,10.000447
1165,student,fold_2,None,30524,0.739362,0.769788,0.133445,0.134311,0.160170,1.895578,0.220440,0.033598,0.033977,8.780742
1166,student,fold_3,None,30524,0.737961,0.753866,0.135213,0.135366,0.168067,1.927140,0.172841,0.030000,0.033494,8.457896
1167,student,fold_4,None,30524,0.752239,0.748526,0.155327,0.156887,0.169593,1.938883,0.213542,0.038373,0.038419,7.659498
1168,action,all_target_data,None,152621,0.658425,0.743430,0.045379,0.048417,0.182685,1.970779,0.218041,0.007114,0.007116,8.719409
1169,assignment,all_target_data,None,152621,0.754579,0.766913,0.161401,0.161402,0.160482,1.921889,0.201550,0.031505,0.031686,8.505214
1170,combined,all_target_data,None,152621,0.769681,0.774081,0.183769,0.183846,0.156201,1.890825,0.189497,0.047677,0.048262,8.363195
1171,student,all_target_data,None,152621,0.743362,0.761402,0.142633,0.142655,0.164073,1.928074,0.205584,0.033379,0.033472,8.488754
